<a href="https://colab.research.google.com/github/ykkim77/nlp_5th/blob/main/word2vec.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Word2Vec 만들기**  <br> 출처 : https://wikidocs.net/50739

In [ ]:
import nltk
nltk.download('punkt')
import urllib.request
import zipfile
from lxml import etree
import re
from nltk.tokenize import word_tokenize, sent_tokenize

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


**훈련 데이터 가져오기**

In [ ]:
# 데이터 다운로드
urllib.request.urlretrieve("https://raw.githubusercontent.com/GaoleMeng/RNN-and-FFNN-textClassification/master/ted_en-20160408.xml", filename="ted_en-20160408.xml")
!ls

!tail -500 ted_en-20160408.xml


sample_data  ted_en-20160408.xml
(Laughter)
I write the code that makes the whole world run. I'm getting royalties from everyone. Sometimes it's garbage, but the press is snowed. You buy the box; I'll sell the code. Every software company is doing Microsoft's R&amp;D. You can't keep a good idea down these days. Even Windows is a hack. We're kind of based loosely on the Mac. So it's big, so it's slow. You've got nowhere to go. I'm not doing this for praise. I write the code that fits the world today. Big mediocrity in every way. We've entered planet domination mode. You'll have no choice; you'll buy my code. I am Bill Gates and I write the code.
(Applause)
But actually, I believe there are really two Microsofts. There's the old one, responsible for Windows and Office. They're dying to throw the whole thing out and start fresh, but they can't. They're locked in, because so many add-ons and other company stuff locks into the old 1982 chassis. But there's also a new Microsoft, that's reall

In [ ]:
re_text = 'test(test11)--eef---(test11112)11111'
re_text1 = 'test<test11>'
print(re.sub(r'\<[^)]*\>', '', re_text1))



test


In [ ]:
targetXML=open('ted_en-20160408.xml', 'r', encoding='UTF8')
target_text = etree.parse(targetXML)


# xml 파일로부터 <content>와 </content> 사이의 내용만 가져온다.
parse_text = '\n'.join(target_text.xpath('//content/text()'))

# 정규 표현식의 sub 모듈을 통해 content 중간에 등장하는 (Audio), (Laughter) 등의 배경음 부분을 제거.
# 해당 코드는 괄호로 구성된 내용을 제거.
content_text = re.sub(r'\([^)]*\)', '', parse_text)


# 입력 코퍼스에 대해서 NLTK를 이용하여 문장 토큰화를 수행.
sent_text = sent_tokenize(content_text)

# 각 문장에 대해서 구두점을 제거하고, 대문자를 소문자로 변환.
normalized_text = []
for string in sent_text:
     tokens = re.sub(r"[^a-z0-9]+", " ", string.lower())
     normalized_text.append(tokens)

# 각 문장에 대해서 NLTK를 이용하여 단어 토큰화를 수행.
result = [word_tokenize(sentence) for sentence in normalized_text]

In [ ]:
print('총 샘플의 개수 : {}'.format(len(result)))

총 샘플의 개수 : 273424


In [ ]:
# 샘플 3개만 출력
for line in result[:3]:
    print(line)

['here', 'are', 'two', 'reasons', 'companies', 'fail', 'they', 'only', 'do', 'more', 'of', 'the', 'same', 'or', 'they', 'only', 'do', 'what', 's', 'new']
['to', 'me', 'the', 'real', 'real', 'solution', 'to', 'quality', 'growth', 'is', 'figuring', 'out', 'the', 'balance', 'between', 'two', 'activities', 'exploration', 'and', 'exploitation']
['both', 'are', 'necessary', 'but', 'it', 'can', 'be', 'too', 'much', 'of', 'a', 'good', 'thing']


**Word2Vec 훈련 시키기**

In [ ]:
from gensim.models import Word2Vec
# size = 워드 벡터의 특징 값. 즉, 임베딩 된 벡터의 차원.window = 컨텍스트 윈도우 크기 min_count = 단어 최소 빈도 수 제한 (빈도가 적은 단어들은 학습하지 않는다.), workers = 학습을 위한 프로세스 수, sg = 0은 CBOW, 1은 Skip-gram.
model = Word2Vec(sentences=result, size=100, window=5, min_count=5, workers=4, sg=0)     


In [ ]:
model_result = model.wv.most_similar("man")
print(model_result)

[('woman', 0.8534528017044067), ('guy', 0.8272244930267334), ('lady', 0.7920387983322144), ('gentleman', 0.7750871777534485), ('boy', 0.7747635841369629), ('girl', 0.7562539577484131), ('soldier', 0.7414239048957825), ('friend', 0.7095862627029419), ('kid', 0.7039577960968018), ('rabbi', 0.6680681705474854)]


**Word2Vec 모델 저장하고 로드하기**

In [ ]:
from gensim.models import KeyedVectors
model.wv.save_word2vec_format('eng_word2vec') # 모델 저장
!ls
loaded_model = KeyedVectors.load_word2vec_format("eng_word2vec") # 모델 로드

eng_word2vec  sample_data  ted_en-20160408.xml


In [ ]:
model_result = loaded_model.most_similar("dog")
print(model_result)

[('cat', 0.8234075903892517), ('chair', 0.783019483089447), ('leg', 0.7752771377563477), ('doctor', 0.7723519802093506), ('seat', 0.7428040504455566), ('mom', 0.7426984906196594), ('nose', 0.7401458024978638), ('pen', 0.7302069067955017), ('uncle', 0.7286360263824463), ('card', 0.7236413955688477)]
